In [1]:
import pandas as pd 
import numpy as np 
import datetime as dt
%matplotlib notebook
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

#### Python  SQL toolkit and ORM

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

#### Read in cleaned measurements and stations csvs

In [23]:
clean_measurements_df = pd.read_csv(r"\Users\victo\Desktop\SQLalchemy\clean_measurements.csv")
clean_measurements_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [24]:
clean_stations_df = pd.read_csv(r"\Users\victo\Desktop\SQLalchemy\clean_stations.csv")
clean_stations_df.head(2)

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6


### merge dataframes to use in later queries 

In [25]:
hawaii_merged_df = pd.merge(clean_measurements_df, clean_stations_df, how="inner", on="station")
hawaii_merged_df.head(2)

,Unnamed: 0_x,station,date,prcp,tobs,Unnamed: 0_y,name,latitude,longitude,elevation
0,0,USC00519397,2010-01-01,0.08,65,0,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00519397,2010-01-02,0.00,63,0,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0


In [6]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

# Reflect Tables into SQLAlchemy ORM 

In [7]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [8]:
connection = engine.connect()

## Create Measurements Table 


In [9]:
Base = declarative_base()

In [10]:
class Measurements(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)  
    tobs = Column(Float)


In [11]:
Base.metadata.create_all(engine)

In [12]:
# Base = automap_base()
# Base.prepare(engine, reflect=True)
# Base.classes.keys()

In [13]:
clean_measurements_df = pd.read_csv(r"\Users\victo\Desktop\SQLalchemy\clean_measurements.csv")
#convert date column to datetime data type to make it easier to handle
clean_measurements_df['date'] = pd.to_datetime(clean_measurements_df['date'], format = '%Y-%m-%d')
clean_measurements_df.head(10)

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70
5,6,USC00519397,2010-01-08,0.00,64
6,7,USC00519397,2010-01-09,0.00,68
7,8,USC00519397,2010-01-10,0.00,73
8,9,USC00519397,2010-01-11,0.01,64
9,10,USC00519397,2010-01-12,0.00,61


In [14]:
clean_measurements_df = clean_measurements_df.dropna()
clean_measurements_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [15]:
# Use Orient='station' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
data_measurements = clean_measurements_df.to_dict(orient='records')

In [16]:
data_measurements[0] #now data is just a bunch of dictionaries for each row 

{'Unnamed: 0': 0,
 'date': Timestamp('2010-01-01 00:00:00'),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [17]:
#http://docs.sqlalchemy.org/en/latest/core/metadata.html
#reflecting tables using metadata from SQLalchemy 
metadata = MetaData(bind=engine)
metadata.reflect()

In [18]:
# Save the reference to the `stations` table as a variable called `station_table`
measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [19]:
#table.delete()` to remove any pre-existing data
connection.execute(measurement_table.delete())

In [20]:
#table.insert()` to insert the data into the table

###populate SQL table
connection.execute(measurement_table.insert(), data_measurements)

####data needs to have NaN's dropped completely####
###TO DO####




In [22]:
### TEST that insertion worked
connection.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

## Create Stations Table

In [26]:
Base = declarative_base()

In [27]:
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)  
    longitude = Column(Float)
    elevation = Column(Float)

In [28]:
Base.metadata.create_all(engine)

In [30]:
clean_stations_df = pd.read_csv(r"\Users\victo\Desktop\SQLalchemy\clean_stations.csv")
clean_stations_df.head(2)

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6


In [32]:
stations_data = clean_stations_df.to_dict(orient='records')
stations_data[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [33]:
#using metadata from SQlalchemy to reflect the new table
metadata = MetaData(bind=engine)
metadata.reflect()

In [35]:
#Save reference to table>>>variable = "station_table"
station_table = sqlalchemy.Table('Stations', metadata, autoload=True)

In [36]:
#table.delete()` to remove any pre-existing data
connection.execute(station_table.delete())

In [37]:
#table.insert()` to insert the data into the table

###populate SQL table
connection.execute(station_table.insert(), stations_data)


In [38]:
#Test that inserting the data worked
connection.execute("Select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]